In [18]:
# Load library
import fairsearchcore as fsc
import pandas as pd
# dataframen = pd.read_csv("ONE_samlet_preds_index_ny_PLAIN.csv")
dataframen = pd.read_csv("TWO_samlet_preds_index_ny_PLAIN.csv")

In [19]:
import numpy as np
dfs = np.array_split(dataframen, 200)
dfs[2]

,Unnamed: 0,Idx,Gender_bins,Preds
8192,0,12063,0,93.552032
8193,1,11714,0,93.552032
8194,2,7332,0,93.552032
8195,3,6991,0,93.552032
8196,4,6641,0,93.552032
...,...,...,...,...
12283,4091,1195,0,22.433973
12284,4092,9931,0,21.522223
12285,4093,1734,0,21.502270
12286,4094,7166,0,21.328962


## NY FA*IR funktion: Maximize utility while maximizing fairness. 




In [20]:
df_try = dataframen#[:10000]
df_try

,Unnamed: 0,Idx,Gender_bins,Preds
0,0,7490,1,92.565254
1,1,12018,1,92.565254
2,2,11864,1,92.565254
3,3,12016,0,92.565254
4,4,1089,0,92.477142
...,...,...,...,...
819195,4091,1195,0,23.099279
819196,4092,1856,0,23.005428
819197,4093,6742,0,23.005428
819198,4094,3588,0,22.907692


In [21]:
# If WO run this
# df_try['Gender_bins'] = df_try['Gender_bins'].str.strip('[]').astype(int)
# df_try

In [22]:
import warnings

In [23]:
def our_FAIR(df, k, n_prefix, prop=-1 ):
    
    """" 
    Our fair algorithm inspired by Zehlike's FA*IR:
    df is predicted df. k is top k ranking from df of length N. n_prefix is desired prop for each n prefix. 
    prop is desired proportion with default value of the proportion of the protected group in the df. 
    
    Output is 'result' of a fair top-k ranking with desired proportion for the whole df and in every prefix
    
    """
    
    if prop<0:
        prop = len(df[df.Gender_bins == 1])/len(df)
        print(prop)
        
    #we subtract by 0, because we are using 0-index
    n_prefix = int(n_prefix *k)-1

    # if k/n_prefix != int(k/n_prefix):
    #     warnings.warn('Please use 0.1 as 10 percent of the top-k as prefix')
    
    #split df in two df groups: protected and non-protected and ensure each of them are sorted
    #in descending order by Preds scores
    
    #protected (1=Female)
    dfp = df[df.Gender_bins == 1].sort_values(by='Preds', ascending=False).reset_index()
    #non-protected
    dfpn = df[df.Gender_bins == 0].sort_values(by='Preds', ascending=False).reset_index()
    
    #dfnew with of the top-k fair sorting of org. df
    #new df
    dfnew = pd.DataFrame(columns = df.columns)
    
    #print OBS if it's not possible to achieve the given prop for each n prefix for the whole top-k
    
    #for-loop for creating ranking in batches of prefix_n through k
    
    #desired number of protected candidates in each n batch
    prop_ok = int(n_prefix*prop)
    prop_pn = int(n_prefix - prop_ok)
    
    #index in each dataframe
    dfp_Idx = 0
    dfpn_Idx = 0
    
    for i in range(int(k/n_prefix)): #loop over number of n_prefixes in length k
        p_Idx = 0
        pn_Idx = 0

        for j in range(n_prefix):
            if dfp.Preds.loc[dfp_Idx] > dfpn.Preds.loc[dfpn_Idx]:
                dfnew = dfnew.append(dfp.loc[dfp_Idx])
                p_Idx += 1
                dfp_Idx += 1
            else:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx])
                pn_Idx += 1
                dfpn_Idx += 1

            #if the proportion of nonprotected group is achieved within the prefix, add rest as protected 
            if pn_Idx == prop_pn:
                dfnew = dfnew.append(dfp.loc[dfp_Idx : dfp_Idx + (prop_ok-p_Idx)])
                dfp_Idx += (prop_ok-p_Idx)
                break
            #if the proportion of protected group is achieved within the prefix, add rest as non-protected
            elif p_Idx == prop_ok:
                dfnew = dfnew.append(dfpn.loc[dfpn_Idx : dfpn_Idx + (prop_pn-pn_Idx) ])
                dfpn_Idx += (prop_pn-pn_Idx)
                break

        dfnew = dfnew.drop(["index"], axis=1).reset_index()
                
    return dfnew



In [24]:
# # tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds'])
# # for i in range(200):
# tom_res = pd.DataFrame()
# for i in range(2):
#     ress = our_FAIR(dfs[i], 0.40, 0.1, 500)
#     # re_ranked_MANGE = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
#     ress.to_csv(f"re_ranked_{i}.csv")

#     # tom_res = tom_res.append(ress)



In [25]:
def fair_mange(df):
    tom_res = pd.DataFrame(columns = ['not_4_use','Idx','Gender_bins','Preds'])
    for i in range(200):
        ress = our_FAIR(dfs[i], 0.40, 0.05, 500)
        # ress.to_csv(f"re_ranked_{i}.csv")
        tom_res = tom_res.append(ress)

    tom_res = tom_res.drop(['index','not_4_use'], axis=1)
    return tom_res.to_csv(f"re_ranked_{i}_NY.csv")
fair_mange(dfs)



In [26]:
# ress = our_FAIR(df_try, 0.40, 0.1, 500)
# sum(ress.Gender_bins[630:700])

Check ratio of women out of total

In [27]:
# # sum(ress.Gender_bins)/len(ress)
# print(ress.Gender_bins.value_counts())
# print(len(ress[ress.Gender_bins == 1]))
# print(len(ress))
# # print(len(ress[ress.Gender_bins == 1])/len(ress))
# print('{0:.05f}'.format(len(ress[ress.Gender_bins == 1])/len(ress)))

## Save this as a reranking and plot the entropy and utility

In [28]:
# re_ranked_WO = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_WO.to_csv("re_ranked_WO.csv")

# re_ranked_PLAIN = pd.DataFrame(ress, columns = ['Idx','Gender_bins','Preds'])
# re_ranked_PLAIN.to_csv("re_ranked_PLAIN.csv")